In [ ]:
!pip install anvil-uplink
from anvil import *
import anvil.server
anvil.server.connect("server_EMHCTR3AO64XZL2LTJR2LX4W-PU76NFNVYXV6QB4N")

from os import pathconf_names
# libraries to deal with data types
import json
import random
import string

# NLP tools that we'll need
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer # our stemmer
nltk.download('punkt') # our tokenizer

# neural network tools we'll need
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dropout, Dense

# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Project Bracket Team 3/intents.json'

with open(path, 'r') as f:
  intents = json.load(f) 


# create our stemmer
stemmer = PorterStemmer()

# creating our lists
all_words = [] # our "dictionary"
tags = [] # list of all our tags
x = [] # list of all our patterns
y = [] # associated tag for each pattern

for intent in intents['intents']:
  for pattern in intent['patterns']:
    # add our patterns and tags to x and y array
    x.append(pattern)
    y.append(intent['tag'])

    # tokenize each pattern and add it to all words array
    tokenizedPattern = nltk.word_tokenize(pattern)
    all_words.extend(tokenizedPattern)

    # add each tag to our general tags array
    if intent["tag"] not in tags:
      tags.append(intent["tag"])

# stemming + lowering + remove punctuation from each word in all words array
all_words = [stemmer.stem(words.lower()) for words in all_words if words not in string.punctuation]

# organize alphabetically (for convenience) and remove duplicates
all_words = sorted(set(all_words))
tags = sorted(set(tags))


# let's train our chatbot
train_data = []
empty = [0] * len(tags)

# creating the bag of words
for index, pattern in enumerate(x):
  bag = []
  add_word = stemmer.stem(pattern.lower())
  for word in all_words:
    bag.append(1) if word in add_word else bag.append(0) # create our array of 1's and 0's

# mark index of the tag to which pattern is associated to
    marked_tag = list(empty)
    marked_tag[tags.index(y[index])] = 1

    train_data.append([bag, marked_tag])

# shuffle  & turn into array
random.shuffle(train_data)
train_data_array = np.array(train_data, dtype=object)

# split the features and target labels
x = np.array(list(train_data_array[:, 0]))
y = np.array(list(train_data_array[:, 1]))

# feed our data into neural network

# parameters
input_nn = (len(x[0]),)
output_nn = len(y[0])
epochs = 200 # number of times algorithm will go through training data

# defining the neural network
model = Sequential()
model.add(Dense(128, input_shape=input_nn, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_nn, activation = "softmax"))

opt = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6) # adding required optimizer for model

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=["accuracy"])

# print(model.summary())
model.fit(x=x, y=y, epochs=200, verbose=0)

def preprocess(msg): 
  tokenized_input = nltk.word_tokenize(msg)
  tokenized_input = [stemmer.stem(w.lower()) for w in tokenized_input]
  return tokenized_input

def bagger(msg, all_words): 
  tokenized_input = preprocess(msg)
  bag_of_words = [0] * len(all_words)
  for wordSingle in tokenized_input: 
    for index, wordWhole in enumerate(all_words):
      if wordWhole == wordSingle: 
        bag_of_words[index] = 1
  return np.array(bag_of_words)

def pred_class(msg, all_words, labels): 
  bag_of_words = bagger(msg, all_words)
  prediction = model.predict(np.array([bag_of_words]))[0]
  thresh = 0.2
  y_prediction = [[idx, res] for idx, res in enumerate(prediction) if res > thresh]

  y_prediction.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_prediction:
    return_list.append(labels[r[0]])
  return return_list

def talk(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json['intents']
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

@anvil.server.callable
def talkLoop(msg):
  msg = msg.lower()
  intents_list = pred_class(msg, all_words, tags)
  result = talk(intents_list, intents)
  return result


anvil.server.wait_forever()











     |████████████████████████████████| 64 kB 1.8 MB/s 
     |████████████████████████████████| 51 kB 168 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=d1ea748022c78ae5b45cc93f6f5e23dd407eda0e4d48f643668996d4d0f07266
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive
